In [ ]:
# Install required libraries
!pip install transformers
!pip install datasets
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

# Log into Hugging Face account
notebook_login()


In [ ]:
from datasets import load_dataset

# Load the EYE-QA-PLUS dataset from Hugging Face Hub
dataset = load_dataset("QIAIUNCC/EYE-QA-PLUS")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/488 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.43M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30152 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4909 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartTokenizer

# Load the BART tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)

# Define the tokenization function with increased max_length (e.g., 256 for both input and output)
def tokenize_function(example):
    input_text = example["input"]    # Use "input" for the question
    target_text = example["output"]  # Use "output" for the answer

    # Tokenize inputs and labels with increased max_length
    model_inputs = tokenizer(input_text, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=256, truncation=True, padding="max_length").input_ids

    # Replace padding token in labels with -100 to ignore during loss calculation
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]
    model_inputs["labels"] = labels
    return model_inputs

# Apply tokenization to the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/30152 [00:00<?, ? examples/s]

Map:   0%|          | 0/4909 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration

# Load the BART model
model = BartForConditionalGeneration.from_pretrained(model_name)


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Set up training arguments with increased generation_max_length (e.g., 256)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_bart",
    evaluation_strategy="no",         # Disable evaluation for faster training
    learning_rate=5e-5,
    per_device_train_batch_size=8,    # Adjust batch size based on your memory capacity
    num_train_epochs=1,               # Set epochs to 1 for quick testing
    weight_decay=0.01,
    logging_steps=10,
    logging_dir="./logs_bart",
    save_steps=1000,                  # Save model every 1000 steps for quicker checkpoints
    save_total_limit=2,               # Keep only the 2 most recent checkpoints to save space
    predict_with_generate=True,       # Enable generation during evaluation
    generation_max_length=256         # Increased max_length for generation
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

# Start training
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,8.259800
20,6.172600
30,4.540100
40,3.708800
50,3.049500
60,2.562200
70,2.145700
80,2.031200
90,1.712200
100,1.852500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=3769, training_loss=1.4685072194173059, metrics={'train_runtime': 2443.1985, 'train_samples_per_second': 12.341, 'train_steps_per_second': 1.543, 'total_flos': 4596193422213120.0, 'train_loss': 1.4685072194173059, 'epoch': 1.0})

In [ ]:
# Save the fine-tuned model and tokenizer (if you haven’t already)
model.save_pretrained("./bart_fine_tuned")
tokenizer.save_pretrained("./bart_fine_tuned")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart_fine_tuned/tokenizer_config.json',
 './bart_fine_tuned/special_tokens_map.json',
 './bart_fine_tuned/vocab.json',
 './bart_fine_tuned/merges.txt',
 './bart_fine_tuned/added_tokens.json')

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the saved model and tokenizer
model = BartForConditionalGeneration.from_pretrained("./bart_fine_tuned")
tokenizer = BartTokenizer.from_pretrained("./bart_fine_tuned")


In [ ]:
def generate_response(prompt):
    # Encode the input text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)

    # Generate the output
    output = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # Decode and return the generated text
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [ ]:
# Test the model with a relevant sample prompt
test_prompt = "What are the common symptoms of viral eye infections?"
print("Prompt:", test_prompt)
print("Response:", generate_response(test_prompt))

# Additional prompts for a thorough evaluation
additional_prompts = [
    "Describe the causes of conjunctivitis.",
    "How can viral eye infections be treated?",
    "What are the common viruses that cause eye infections?",
    "Can viral eye infections lead to complications?",
    "How is conjunctivitis diagnosed in a clinical setting?",
    "What precautions should be taken to prevent eye infections?"
]

# Print responses for each relevant test prompt
for prompt in additional_prompts:
    print("\nPrompt:", prompt)
    print("Response:", generate_response(prompt))


Prompt: What are the common symptoms of viral eye infections?
Response: Viral eye infections can present with various symptoms. Some common symptoms include blurred vision, redness, sensitivity to light, and redness of the eyes. Other common symptoms may include redness and sensitivity to bright light. It is important for ophthalmologists to be aware of these symptoms when treating patients with viral eye infections.

Prompt: Describe the causes of conjunctivitis.
Response: Conjunctivitis can be caused by various factors. One of the most common causes is the presence of conjunctiva, which is a thick, yellowish-white substance that fills the space between the conjunctival surface and the cornea. Conjunctiva can also be associated with other conditions, such as conjunctibacterial infections, corneal ulceration, and inflammation. The most common cause is the use of antibiotics, which can help reduce inflammation and promote healing. It is important to consult with an ophthalmologist to de

In [ ]:
from huggingface_hub import notebook_login

# Log into Hugging Face (if you haven't already)
notebook_login()


In [ ]:
from huggingface_hub import HfApi

# Initialize HfApi
api = HfApi()

# Define a new repository name with your username
repo_id = "keshish/bart-eye-qa-v2"  # Change "bart-eye-qa-v2" if needed


In [ ]:
# Create the repository on Hugging Face
api.create_repo(repo_id=repo_id, exist_ok=True)


RepoUrl('https://huggingface.co/keshish/bart-eye-qa-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='keshish/bart-eye-qa-v2')

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./bart_fine_tuned")
tokenizer.save_pretrained("./bart_fine_tuned")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart_fine_tuned/tokenizer_config.json',
 './bart_fine_tuned/special_tokens_map.json',
 './bart_fine_tuned/vocab.json',
 './bart_fine_tuned/merges.txt',
 './bart_fine_tuned/added_tokens.json')

In [ ]:
# Push model and tokenizer to the new repository on Hugging Face
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"Model and tokenizer successfully uploaded to https://huggingface.co/{repo_id}")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model and tokenizer successfully uploaded to https://huggingface.co/keshish/bart-eye-qa-v2
